In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df_original = pd.read_csv("données sans doublon.csv")
df = df_original[['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges']]

df = df[df['charges'] < 49000]

display(df)

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1332,50,male,30.970,3,no,northwest,10600.54830
1333,18,female,31.920,0,no,northeast,2205.98080
1334,18,female,36.850,0,no,southeast,1629.83350
1335,21,female,25.800,0,no,southwest,2007.94500


___
MODELE AVEC PIPELINE
___

In [2]:
df_original = pd.read_csv("data_cleaned_cheated.csv")
df = df_original[['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges', 'charges_group_cheat','bmi_category']]

# df.loc[:, 'charges_group'] = np.where(df['smoker'] == 0, 1,
#                                       np.where(df['bmi'] >= 30, 3,
#                                                np.where(df['sex'] == 0, 2,
#                                                         np.where(df['age'] < 22, 1, 3))))
df.loc[:, 'charges_group'] = np.where(df['smoker'] == 0, 1,
                                      np.where(df['bmi'] >= 30, 3,
                                               2))
df['have_children'] = np.where(df['children'] > 0, 1, 0)

# nettoyage
df = df[['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges', 'charges_group', 'have_children','bmi_category']]

display(df.head())

,age,sex,bmi,children,smoker,region,charges,charges_group,have_children,bmi_category
0,19,1,27.900,0,1,southwest,16884.92400,2,0,Surpoids
1,18,0,33.770,1,0,southeast,1725.55230,1,1,Obésité modérée
2,28,0,33.000,3,0,southeast,4449.46200,1,1,Obésité modérée
3,33,0,22.705,0,0,northwest,21984.47061,1,0,Poids normal
4,32,0,28.880,0,0,northwest,3866.85520,1,0,Surpoids


In [3]:
from sklearn.model_selection import train_test_split # pour diviser le jeu de données initial en 1 jeu d'entrainement (X_train, y_train) et un jeu de test (X_test, y_test)
from sklearn.model_selection import GridSearchCV
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
import joblib

y = df[['charges']]

X = df[['age', 'sex', 'bmi', 'children', 'smoker', 'region']]
X['smoker_bmi'] = X['smoker'] * X['bmi']
X['smoker_age'] = X['smoker'] * X['age']
X['smoker_children'] = X['smoker'] * X['children']
# display(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42, stratify=X['smoker'])

numerical_features = make_column_selector(dtype_include=np.number) # plutôt que : numerical_features = ['age', 'bmi', 'children']
categorical_features = make_column_selector(dtype_exclude=np.number) # plutôt que : categorical_features = ['region']

numerical_pipeline = make_pipeline(SimpleImputer(), StandardScaler())
categorical_pipeline = make_pipeline(SimpleImputer(strategy='most_frequent'), OneHotEncoder())

preprocessor = make_column_transformer((numerical_pipeline, numerical_features),
                                       (categorical_pipeline, categorical_features))


print("##################################################")
print("##################################################")
print("##################################################")
# AVEC GRID SEARCH
# Meilleurs hyperparamètres :  {'lassocv__alphas': [2, 1, 1, 1, 1, 1, 1, 1], 'lassocv__cv': 3, 'lassocv__eps': 0.1, 'lassocv__n_alphas': 100, 'lassocv__random_state': 42}
# 'lassocv__cv': 3, 'lassocv__eps': 0.1, 'lassocv__n_alphas': 100, 'lassocv__random_state': 42}

model = make_pipeline(preprocessor, LassoCV(alphas=[2, 1, 1, 1, 1, 1, 1, 1], cv=3, eps=0.1, n_alphas=100, random_state=42))
model.fit(X_train, y_train)
print("Avec GRID >", round(model.score(X_test, y_test)*100, 2))
model = make_pipeline(preprocessor, LassoCV(random_state=42))

# SANS GRID SEARCH
model.fit(X_train, y_train)
print("Premier modèle >", round(model.score(X_test, y_test)*100, 2))

# sauvegarde du meilleur modèle
joblib.dump(model, "model.sav")



# GRID SEARCH
if True == False:
    param_grid = {
        'lassocv__eps': [10**-x for x in range(1,6)],
        'lassocv__n_alphas': [100 * x for x in range(1,4)],
        'lassocv__alphas': [[1,1,1,1,1,1,1,1],
                            [2,1,1,1,1,1,1,1],
                            [1,2,1,1,1,1,1,1],
                            [1,1,2,1,1,1,1,1],
                            [1,1,1,2,1,1,1,1],
                            [1,1,1,1,2,1,1,1],
                            [1,1,1,1,1,2,1,1],
                            [1,1,1,1,1,1,2,1],
                            [1,1,1,1,1,1,1,2]],
        'lassocv__cv': [x for x in range(3,8)],
        'lassocv__random_state': [42]
    }
    grid_search = GridSearchCV(model, param_grid)
    # Entraînement du modèle avec recherche de grille
    grid_search.fit(X_train, y_train)

    print("Meilleurs hyperparamètres : ", grid_search.best_params_)
    # grid_search.predict(X_test))

/tmp/ipykernel_23816/1764717075.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['smoker_bmi'] = X['smoker'] * X['bmi']
/tmp/ipykernel_23816/1764717075.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['smoker_age'] = X['smoker'] * X['age']
/tmp/ipykernel_23816/1764717075.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

##################################################
##################################################
##################################################
Avec GRID > 90.11
Premier modèle > 90.29
